# Imports

In [1]:
import os
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import random_split
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm

# Loader

In [2]:
CLASSES = ["wearing", "packaging", "drinking", "passing"]

class IMUDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.samples = self._prepare_dataset()

    def _prepare_dataset(self):
        samples = []
        for dir_name in os.listdir(self.root_dir):
            label = dir_name.split('_')[0]
            full_path = os.path.join(self.root_dir, dir_name)
            if os.path.isdir(full_path):
                samples.append((full_path, label))
        return samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label_str = self.samples[idx]
        imu_data = pd.read_csv(os.path.join(path, 'out.csv'))

        # Select only the IMU data columns (assuming first 6 are IMU related)
        imu_data = imu_data.iloc[:, :6]  

        # Convert label string to numeric
        label = CLASSES.index(label_str)

        # Convert IMU data and label to tensors
        imu_tensor = torch.tensor(imu_data.values, dtype=torch.float32)
        label_tensor = torch.tensor(label, dtype=torch.long)

        sample = {
            'imu_data': imu_tensor,
            'label': label_tensor
        }
        return sample

# Model

In [3]:
class IMUOnlyClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(IMUOnlyClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x shape: (batch_size, sequence_length, input_size)
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Get the last time step
        return out

# Split

In [4]:
dataset = IMUDataset(root_dir='../data/')

# Assume 'dataset' is your initialized PyTorch dataset
dataset_size = len(dataset)
train_size = int(0.75 * dataset_size)
test_size = dataset_size - train_size

# Split the dataset into train and test sets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# You can now use these subsets with a DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

# Train

In [5]:
input_size = 6  # For example, 3 axes for gyro and 3 for accel
hidden_size = 128  # Example size, can be tuned
num_layers = 1  # Number of LSTM layers
num_classes = 4

model = IMUOnlyClassifier(input_size, hidden_size, num_layers, num_classes)

In [6]:
# Assuming model is already defined and initialized
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10  # Define the number of epochs
for epoch in tqdm(range(num_epochs)):
    model.train()  # Set the model to training mode
    for i, data in enumerate(train_loader):
        # Forward pass
        outputs = model(data['imu_data'])
        loss = criterion(outputs, data['label'])
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

c:\Users\Lukas Volk\Documents\ubicomp-capstone\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 10/10 [00:01<00:00,  7.08it/s]


# Eval

In [8]:
# Function to convert tensors to numpy arrays
def to_numpy(tensor):
    return tensor.cpu().detach().numpy()

# Evaluation
model.eval()
all_labels = []
all_predictions = []

with torch.no_grad():
    for i, data in enumerate(test_loader):
        # Forward pass
        outputs = model(data['imu_data'])
        _, predicted = torch.max(outputs, 1)
        
        all_labels.extend(to_numpy(data['label']))
        all_predictions.extend(to_numpy(predicted))

# Convert all labels and predictions to numpy arrays for sklearn functions
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

# Calculate and print metrics
print(classification_report(all_labels, all_predictions))

# Calculate and print the confusion matrix
conf_matrix = confusion_matrix(all_labels, all_predictions)
print("Confusion Matrix:")
print(conf_matrix)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10

Confusion Matrix:
[[3 0 0 0]
 [0 3 0 0]
 [0 0 2 0]
 [0 0 0 2]]


# Save

In [9]:
# Save the entire model
torch.save(model, '../model/imu_only_rnn.pth')

# Load

In [10]:
# To load the model later
loaded_model = torch.load('../model/imu_only_rnn.pth')